In [1]:
import pandas as pd
import numpy as np
import faiss
import catboost as cb
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


In [2]:
df_base = pd.read_csv("data/base.csv", index_col=0)
df_base.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,181.449700,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,195.234190,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,164.381470,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,136.233580,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,112.119064,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


In [3]:
# Scale data
scaler = StandardScaler()
scaled_vectors = pd.DataFrame(scaler.fit_transform(df_base), index=df_base.index)

In [4]:
# number of dimensions and clusters
dim = scaled_vectors.shape[1]
k_clusters = 1024

# init 
quantizer = faiss.IndexFlatL2(dim)
idx_l2 = faiss.IndexIVFFlat(quantizer, dim, k_clusters)

# number of clusters to check
idx_l2.nprobe = 16

# Number of neighbors to search
k_neighbors = 100

In [5]:
idx_l2.train(np.ascontiguousarray(scaled_vectors.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(scaled_vectors.values).astype('float32'))

In [6]:
# Сохраним базовые индексы
base_index = {k: v for k, v in enumerate(scaled_vectors.index.to_list())}

In [7]:
df_train = pd.read_csv("data/train.csv", index_col=0)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,Target
Id,,,,,,,,,,,,,,,,,,,,,
0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,30.413513,...,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,140.331070,...,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,211.607820,...,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,242.650650,...,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,103.392670,...,45.028910,-196.09207,-117.626337,66.926220,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base


In [8]:
targets = df_train["Target"]
df_train.drop("Target", axis=1, inplace=True)

In [9]:
# Нормализация векторов
scaled_train = pd.DataFrame(scaler.transform(df_train), index=df_train.index)

In [10]:
# Получим расстояния и индексы соседей
dist, idx = idx_l2.search(np.ascontiguousarray(scaled_train.values).astype('float32'), k_neighbors)

In [11]:
# Преобразуем индекс к np.array
idx = np.array([[base_index[_] for _ in el] for el in idx], dtype=object)

In [12]:
# Посчитаем acc@5
acc = 0
for target, el in zip(targets.values.tolist(), idx):
    if target in el:
        acc += 1

print(100 * acc / len(idx))

76.077


In [13]:
# Сохраним целевой признак для catboost
cb_target = np.array([[1 if x == target else 0 for x in el] for el, target in zip(idx, targets.values.tolist())])

In [14]:
print('dist:', dist.shape)
print('idx:', idx.shape)
print('targets:', targets.shape)

dist: (100000, 100)
idx: (100000, 100)
targets: (100000,)


In [15]:
# Clear memory
del base_index
del df_train
del df_base

In [16]:
# Получение признаков кандидатов и истинного соседа
candidate_features = scaled_vectors.loc[idx.flatten()].values
neighbor_features = scaled_vectors.loc[targets].values

# Изменение порядка расстояний
reshaped_dist = dist.reshape(-1,1)

# Повторение значений признаков для таргета
repeated_neighbor = np.repeat(neighbor_features, k_neighbors, axis=0)

# Проверка размерностей
print('dist:', reshaped_dist.shape)
print('idx:', candidate_features.shape)
print('targets:', repeated_neighbor.shape)

# Горизонтальное объединение массивов
cb_features = np.hstack((reshaped_dist, candidate_features, repeated_neighbor))
print('CB_features:', cb_features.shape)

dist: (10000000, 1)
idx: (10000000, 72)
targets: (10000000, 72)
CB_features: (10000000, 145)


In [17]:
# Clear memory
del scaled_vectors
del neighbor_features
del reshaped_dist
del repeated_neighbor
del candidate_features
del idx
del dist
del scaled_train
del targets